# Payer Model with XGBoost
_**Using Gradient Boosted Trees to Predict Client Insurance Payers**_

---
---

## Contents


In [1]:
import sys

!{sys.executable} -m pip install sagemaker pandas numpy xgboost --upgrade

     |████████████████████████████████| 513 kB 7.7 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 173.6 MB 4.0 kB/s             
  Created wheel for sagemaker: filename=sagemaker-2.77.1-py2.py3-none-any.whl size=711661 sha256=bd7e07b638df857ebcf36a336522d87e3cde6e8e17e8949b872009d94a46b131
  Stored in directory: /home/ec2-user/.cache/pip/wheels/dc/d8/d1/2e5e90c759d53e76964a7eec7b651107a5e430ec5f04caddda
Successfully built sagemaker
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.74.0
    Uninstalling sagemaker-2.74.0:
      Successfully uninstalled sagemaker-2.74.0


In [175]:
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
prefix = "sagemaker/WiEco-Payer-Model"

# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

In [159]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
import sys
import time
import json
from IPython.display import display
from time import strftime, gmtime
from sagemaker.inputs import TrainingInput
from sagemaker.serializers import CSVSerializer

---
## Data Processing

In [161]:
# MotionMD Data
momd_pa_types = {
    'momd_pa' : str,
    'momd_clinic_id' : str,
    'momd_clinic_zip' : str,
    'momd_payer_name' : str,
    'momd_payer_position' : str,
    'momd_payer_type' : str,
    'momd_insureds_policy_prefix' : str
}

momd_pa_cols = [
    'momd_pa',
    'momd_clinic_id',
    'momd_clinic_zip',
    'momd_payer_name',
    'momd_payer_position',
    'momd_payer_type',
    'momd_insureds_policy_prefix'
]

momd_df = pd.read_csv("s3://nathan.ml/mike.data.dump/2021-02-24-MotionMD.csv", dtype=momd_pa_types, usecols=momd_pa_cols)

In [162]:
momd_df

,momd_pa,momd_clinic_id,momd_clinic_zip,momd_payer_name,momd_payer_position,momd_payer_type,momd_insureds_policy_prefix
0,4139454,295247,33542,WELLCARE URGENT CARE,1,COMMERCIAL,160
1,4356998,621901,33544,SIMPLY MEDICAID HMO URGENT CARE,1,MEDICAID,717
2,2908431,295247,33542,UHC DUAL CHOICE PRIMARY PPO/HMO/MEDICARE SOLUTION,1,MEDICARE,114
3,2908431,295247,33542,UHC DUAL CHOICE SECONDARY PPO/HMO,2,COMMERCIAL,114
4,2910494,295247,33542,UHC MEDICARE SOLUTIONS ADVANTAGE PPO,1,MEDICARE,966
...,...,...,...,...,...,...,...
843619,5387846,3577632,83301,BC/BS OUT OF STATE PPO,1,COMMERCIAL,AQK
843620,5388086,3577632,83301,BLUE CROSS OF IDAHO PPO,1,COMMERCIAL,XMF
843621,5106501,3577632,83301,SLHP UHC MEDICARE ADVANTAGE,1,COMMERCIAL,918
843622,4986734,3577632,83301,BLUE CROSS BLUE SHIELD,1,COMMERCIAL,123


In [163]:
# DataWorks Data

dw_pa_cols = {
    'Ppa',
    'Number'
}

dw_pa_types = {
    'Ppa': str,
    'Number': str
}

#dw_df = pd.read_csv("s3://nathan.ml/mike.data.dump/2021-02-24-Dataworks.csv", usecols=dw_pa_cols, dtype=dw_pa_types)
dw_df = pd.read_csv("s3://nathan.ml/mike.data.dump/2021-02-24-Dataworks.csv", 
                    encoding='utf-16', 
                    delim_whitespace=True,
                    usecols=dw_pa_cols,
                    dtype=dw_pa_types)
dw_df = dw_df.rename(columns={'Ppa':'ppa_number', 'Number':'payer_id'})

In [164]:
dw_df

,ppa_number,payer_id
0,Z34685002345,D3982
1,Y00000049858,D88
2,T30584700419,D8602
3,T28249102882,D107
4,O00000045411,D218
...,...,...
555704,1679247,D128
555705,1588042,D138
555706,1573786,UNK
555707,1475870,D138


In [165]:
df = pd.merge(momd_df, dw_df, left_on=["momd_pa"], right_on=["ppa_number"], how='inner')
df

,momd_pa,momd_clinic_id,momd_clinic_zip,momd_payer_name,momd_payer_position,momd_payer_type,momd_insureds_policy_prefix,ppa_number,payer_id
0,2908431,295247,33542,UHC DUAL CHOICE PRIMARY PPO/HMO/MEDICARE SOLUTION,1,MEDICARE,114,2908431,UNK
1,2908431,295247,33542,UHC DUAL CHOICE SECONDARY PPO/HMO,2,COMMERCIAL,114,2908431,UNK
2,2915859,295247,33542,AVMED,1,COMMERCIAL,A53,2915859,D8870
3,2922040,295247,33542,MEDICARE PART B,1,MEDICARE,5K9,2922040,M0003
4,2922040,295247,33542,WPS TRICARE FOR LIFE,2,COMMERCIAL,000,2922040,M0003
...,...,...,...,...,...,...,...,...,...
486108,5230952,3577632,83301,MEDICARE PART A \T\ B,1,MEDICARE,4VW,5230952,M0004
486109,5230952,3577632,83301,TRICARE FOR LIFE/WPS,2,COMMERCIAL,535,5230952,M0004
486110,5244999,3577632,83301,SLHP UHC MEDICARE ADVANTAGE,1,COMMERCIAL,977,5244999,D5172
486111,5274425,3577632,83301,BC/BS OUT OF STATE PPO,1,COMMERCIAL,AQK,5274425,D124


In [166]:
#Move Dependent Variable Column to Front
df = df[ ['payer_id'] + [ col for col in df.columns if col != 'payer_id' ] ]
# Get rid of duplicate id column
df = df.iloc[:,:-1]
df = df.dropna(axis=0)

#Data Cleanup (from Moana)
TOLERANCE = 10 # Lower values are more forgiving

print(df.shape, df['payer_id'].nunique())

numm = pd.DataFrame(df.groupby('payer_id').size(),columns = ['size'])
df = pd.merge(df,numm,left_on = 'payer_id', right_on = numm.index)
df = df.loc[df['size']>=TOLERANCE]
df.drop(columns=['size'],inplace=True)

print(df.shape, df['payer_id'].nunique())

df

(486066, 8) 1012
(484755, 8) 638


,payer_id,momd_pa,momd_clinic_id,momd_clinic_zip,momd_payer_name,momd_payer_position,momd_payer_type,momd_insureds_policy_prefix
0,UNK,2908431,295247,33542,UHC DUAL CHOICE PRIMARY PPO/HMO/MEDICARE SOLUTION,1,MEDICARE,114
1,UNK,2908431,295247,33542,UHC DUAL CHOICE SECONDARY PPO/HMO,2,COMMERCIAL,114
2,UNK,3223847,295247,33542,AETNA PPO/POS,1,COMMERCIAL,W17
3,UNK,3945007,295247,33542,UHC GROUP MEDICARE ADVANTAGE PPO,1,MEDICARE,972
4,UNK,4037998,295247,33542,AARP/UHC MEDICARE COMPLETE/HMO/PPO,1,MEDICARE,903
...,...,...,...,...,...,...,...,...
486061,D323,5163267,360389,38242,WINDSOR,1,COMMERCIAL,316
486062,D323,4726794,360389,38242,WINDSOR,1,COMMERCIAL,313
486063,D323,4726794,360389,38242,MEDICAID/BLUECARE,2,MEDICAID,TD5
486064,D323,4886468,360389,38242,WINDSOR,1,COMMERCIAL,182


---
## Encoding of Model
### Done so the data can be trained using XGBoost

In [167]:
from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder
# Encode string class values as integers (needed for XGBoost)
le = LabelEncoder()
df_encoded = df
for col in df:
    df_encoded[col] = le.fit_transform(df[col])
df_encoded

,payer_id,momd_pa,momd_clinic_id,momd_clinic_zip,momd_payer_name,momd_payer_position,momd_payer_type,momd_insureds_policy_prefix
0,637,3356,193,305,10952,0,3,148
1,637,3356,193,305,10953,4,1,148
2,637,43569,193,305,792,0,1,13943
3,637,137890,193,305,10976,0,3,5908
4,637,149908,193,305,306,0,3,5836
...,...,...,...,...,...,...,...,...
486061,280,313607,599,392,12716,0,1,1589
486062,280,251571,599,392,12716,0,1,1586
486063,280,251571,599,392,7665,4,2,13090
486064,280,274949,599,392,12716,0,1,230


In [168]:
payer_id = df_encoded.iloc[:, 0]
print(f'NumClasses: {len(np.unique(payer_id))}')

NumClasses: 638


In [169]:
# get the training and validation data with a 80% split
train_data, validation_data = np.split(df.sample(frac=1, random_state=1077), [int(0.8 * len(df))]) 
train_data.shape, validation_data.shape

((387804, 8), (96951, 8))

---
## Training in AWS

In [170]:
# turn the training and testing data into csv files
train_data.to_csv('train.csv', header=False, index=False)
validation_data.to_csv('validation.csv', header=False, index=False)

In [171]:
# upload training and testing data to bucket
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')

In [172]:
#Grab XGBoost container
container = sagemaker.image_uris.retrieve('xgboost', boto3.Session().region_name, '1')
display(container)

'811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:1'

In [176]:
s3_input_train = TrainingInput(
    s3_data="s3://{}/{}/train".format(bucket, prefix), content_type="csv"
)
s3_input_validation = TrainingInput(
    s3_data="s3://{}/{}/validation/".format(bucket, prefix), content_type="csv"
)

In [190]:
common_training_params = \
{
    "AlgorithmSpecification": {
        "TrainingImage": container,
        "TrainingInputMode": "File"
    },
    "RoleArn": role,
    "OutputDataConfig": {
        "S3OutputPath": "s3://{}/{}/ouput".format(bucket, prefix)
    },
    "ResourceConfig": {
        "InstanceCount": 1,   
        "InstanceType": "ml.m4.xlarge",
        "VolumeSizeInGB": 10
    },
    "HyperParameters": {
        "max_depth":"40",
        "eta":"0.02",
        "num_round":"200",
        "sample_type":"weighted",
        "silent":"0",
        "subsample":"0.8",
        "objective": "multi:softprob",
        "num_class": str(len(df['payer_id'].unique()))
    },
    "StoppingCondition": {
        "MaxRuntimeInSeconds": 3600*30
    },
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": "s3://{}/{}/train".format(bucket, prefix),
                    "S3DataDistributionType": "FullyReplicated" 
                }
            },
            "ContentType": "csv",
            "CompressionType": "None"
        },
        {
            "ChannelName": "validation",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": "s3://{}/{}/validation".format(bucket, prefix),
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "ContentType": "csv",
            "CompressionType": "None"
        }
    ]
}

In [191]:
training_job_name = 'training-job' 
print("Job name is:", training_job_name)
import copy
training_job_params = copy.deepcopy(common_training_params)
training_job_params['TrainingJobName'] = training_job_name

Job name is: training-job


In [192]:
%%time
#Run Training Job
sm = boto3.Session(region_name=boto3.Session().region_name).client('sagemaker')
sm.create_training_job(**training_job_params)

CPU times: user 211 ms, sys: 8.79 ms, total: 219 ms
Wall time: 413 ms


{'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:173971985270:training-job/training-job',
 'ResponseMetadata': {'RequestId': '6178ba2b-0cb5-4d86-996a-85d639088662',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '6178ba2b-0cb5-4d86-996a-85d639088662',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '87',
   'date': 'Fri, 04 Mar 2022 23:50:14 GMT'},
  'RetryAttempts': 0}}

In [201]:
#Status/Warnings
status = sm.describe_training_job(TrainingJobName=training_job_name)['TrainingJobStatus']
while status=='InProgress':
    status = sm.describe_training_job(TrainingJobName=training_job_name)['TrainingJobStatus']
    print("Status: " + status)
    time.sleep(60)

Status: InProgress


KeyboardInterrupt: 

In [2]:
#Status/Warnings
import boto3
training_job_name = 'training-job' 
sm = boto3.Session(region_name=boto3.Session().region_name).client('sagemaker')
status = sm.describe_training_job(TrainingJobName=training_job_name)['TrainingJobStatus']
while status=='InProgress':
    status = sm.describe_training_job(TrainingJobName=training_job_name)['TrainingJobStatus']
    print("Status: " + status)
    #time.sleep(60)

Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InPr

KeyboardInterrupt: 